In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import pandas as pd
import numpy as np
import ast
import csv
import cugraph
import cudf
import asyncio
import time
import seaborn as sns
import matplotlib.pyplot as plt

/usr/local/lib/python3.11/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained("DeepSeek-Prover-V2-7B")
model = AutoModelForCausalLM.from_pretrained("DeepSeek-Prover-V2-7B")

`rope_scaling`'s factor field must be a float >= 1, got 16
`rope_scaling`'s beta_fast field must be a float, got 32
`rope_scaling`'s beta_slow field must be a float, got 1
Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.14s/it]


In [3]:
model.to('cuda')

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(102400, 4096)
    (layers): ModuleList(
      (0-29): 30 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-06)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-06)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-06)
    (rotary_

## Upload dataset

In [4]:
mtd0_df = pd.read_csv('data/correct_incorrect_df.csv')
# датасет так составлен, что
# правильных и неправильных доказательств будет поровну
# 10'000 считается 3 дня
mtd0_df = mtd0_df[:10000]
mtd0_df.head()

,data_id,proof,is_correct,proof_len,proof_part
0,10137740601197937323927000320,['p1 p2 p3 p4 p5 : Prop\n⊢ ((False ∨ ((((p4 ∧ ...,True,7,['p1 p2 p3 p4 p5 : Prop\n⊢ ((False ∨ ((((p4 ∧ ...
1,101901011237190232879400816388,['p1 p2 p3 p4 p5 : Prop\n⊢ ((((p5 → p1) → (((p...,False,8,['p1 p2 p3 p4 p5 : Prop\n⊢ ((((p5 → p1) → (((p...
2,10147402779818504157948999715,['p1 p2 p3 p4 p5 : Prop\n⊢ ((False ∨ (p4 → p2 ...,True,9,['p1 p2 p3 p4 p5 : Prop\n⊢ ((False ∨ (p4 → p2 ...
3,102028860579639362150235400604,['p1 p2 p3 p4 p5 : Prop\n⊢ ((((((((True ∧ p1) ...,False,11,['p1 p2 p3 p4 p5 : Prop\n⊢ ((((((((True ∧ p1) ...
4,101865771099929866782122861458,['p1 p2 p3 p4 p5 : Prop\n⊢ (((p2 ∧ ((False → F...,True,8,['p1 p2 p3 p4 p5 : Prop\n⊢ (((p2 ∧ ((False → F...


In [5]:
from transformers import AutoConfig
config = AutoConfig.from_pretrained("DeepSeek-Prover-V2-7B")
print("n_layer:", config.num_hidden_layers)
print("n_head:", config.num_attention_heads)

print("Data length", len(mtd0_df))

`rope_scaling`'s factor field must be a float >= 1, got 16
`rope_scaling`'s beta_fast field must be a float, got 32
`rope_scaling`'s beta_slow field must be a float, got 1


n_layer: 30
n_head: 32
Data length 10000


In [6]:
n_layers = 30
n_heads = 32
l_q = []
for i in range(n_layers):
  for j in range(n_heads):
    l_q.append(f'{i} {j}')

mtd0_df[l_q] = np.nan
self_att_df = mtd0_df.copy()

/tmp/ipykernel_21804/1759731962.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  mtd0_df[l_q] = np.nan
/tmp/ipykernel_21804/1759731962.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  mtd0_df[l_q] = np.nan
/tmp/ipykernel_21804/1759731962.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  mtd0_

In [7]:
def mask_attention_within_lines(attentions, token_indices):
    """
    Обнуляет внимание внутри каждой строки в матрице внимания.
    """
    # Создаём маску внимания внутри каждой строки
    for start_idx, end_idx in token_indices:
        # Зануляем внимание внутри строки
        attentions[:, 0, start_idx:end_idx, start_idx:end_idx] = 0

    return attentions

In [8]:
async def process_layer(L, attentions):
    scores = {}
    for Q, layer_att in enumerate(attentions[L][0]):

        src, dst = torch.nonzero(layer_att, as_tuple=True)  # Только ненулевые рёбра
        weights = layer_att[src, dst]  # Веса рёбер

        # Конвертируем в cuDF (GPU-аналог pandas)
        df = cudf.DataFrame({
            "src": src.int().tolist(),
            "dst": dst.int().tolist(),
            "weight": weights.tolist()
        })
        num_nodes = layer_att.shape[0]

        # Создаём граф на GPU
        G = cugraph.Graph()
        G.from_cudf_edgelist(
            df,
            source="src",
            destination="dst",
            edge_attr="weight"
        )

        # Строим минимальное остовное дерево (MST) на GPU
        T = cugraph.minimum_spanning_tree(G)

        # Средний вес рёбер MST
        T_df = T.view_edge_list()  # Получаем DataFrame с рёбрами MST
        sum_weight = T_df["weight"].sum()
        scores[f"{L} {Q}"] = sum_weight / len(T_df)

    return scores

In [9]:
async def all_mtd0(attentions, token_indices):
    full_scores = {}
    attentions = torch.stack(attentions[0])

    attentions = mask_attention_within_lines(attentions, token_indices)
    attentions = (attentions + attentions.transpose(3, 4)) / 2.0  #symmetrization

    tasks = [process_layer(L, attentions) for L in range(n_layers)]
    results = await asyncio.gather(*tasks)

    for scores in results:
        full_scores.update(scores)

    return full_scores

In [10]:
def self_attention(attentions, token_indices):
    attentions = torch.stack(attentions[0])
    score_dict = {}

    for layer_head in l_q:
        layer, head = list(map(int, layer_head.split()))
        score = 0
        for part_ids in token_indices:
            score += attentions[layer][0][head][part_ids[1] - 1][part_ids[1] - 1]
        score /= len(token_indices)
        score_dict[layer_head] = float(score)
    return score_dict


In [11]:
def get_indices_and_encode(statement_list, tokenizer, prompt=""):
    """
    Возвращает индексы токенов начала и конца каждой строки из statement_text.

    :param statement_list: список состояний и тактик доказательства
    :param tokenizer: токенизатор модели
    :param prompt: промпт
    :return: список (start_token_idx, end_token_idx) для каждой строки
    """
    input_list = [prompt] + statement_list
    # чтобы у всех строк на конце был \n
    for i in range(2, len(input_list)):
        input_list[i] += '\n'

    token_indices_list = []
    for i, part in enumerate(input_list):
        encoded_part = tokenizer(part)
        token_len = len(encoded_part['input_ids'])
        if i == 0:
            token_indices_list.append((0, token_len))
        else:
            # минус 1, чтобы не учитывать <｜begin▁of▁sentence｜>
            token_indices_list.append((
                token_indices_list[-1][1],
                token_indices_list[-1][1] + token_len - 1
            ))

    input_text = ''.join(map(str, input_list))
    encoded_input = tokenizer(
        input_text,
        return_tensors="pt"
    )

    return token_indices_list, encoded_input

In [ ]:
prompt = f"Make sure the proof is correct.\n"

for index, row in mtd0_df.iterrows():
    if index % 50 == 0:
        print(f"Processed {index} / {len(mtd0_df)}, time: {time.time()}")

    statement_list = ast.literal_eval(row["proof"])
    token_indices, encoded_input = get_indices_and_encode(
        statement_list,
        tokenizer,
        prompt
    )
    output = model.generate(**encoded_input.to('cuda'),
        max_new_tokens=1,
        output_attentions=True,
        return_dict_in_generate=True,
        pad_token_id=tokenizer.eos_token_id
    )

    # self att
    self_att_row = self_attention(
        output.attentions,
        token_indices
    )
    self_att_df.loc[index, self_att_row.keys()] = self_att_row.values()

    # mtd0
    mtd0_row = await all_mtd0(
        output.attentions,
        token_indices
    )
    mtd0_df.loc[index, mtd0_row.keys()] = mtd0_row.values()

    if index == 200 or (index >= 500 and index % 500 == 0):
        mtd0_df[:index].to_csv(f'results/mtd0_df_{index}_df.csv', index=False)
        self_att_df[:index].to_csv(f'results/self_att_df_{index}_df.csv', index=False)

Processed 0 / 10000, time: 1747147170.682845


`torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True`. Falling back to eager attention. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.
